In [3]:
import tensorflow as tf
import pickle
import numpy as np
import timeit
import load_cifar
import tensorflow.keras as keras
from load_cifar import preprocess_data
import matplotlib.pyplot as plt
import keras_tuner as kt

In [ ]:
tf.reset_default_graph()

<h1>Define Hyper-perparmeter</h1>

<h1>Define Placeholder</h1>

<h1>Define Neural Network Architecture</h1>

In [ ]:
def mnist_model(hp):
    """
    define model for hyper tune
    """
    activation = hp.Choice("activation", values=['relu', 'tanh', 'sigmoid'])

    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(32,32,3)))
    for i in range(hp.Int("num_layers", 2, 6)):
        model.add(
            keras.layers.Dense(
                units=hp.Int("units_" + str(i), min_value=16, max_value=256, step=16),
                activation=activation,
            )
        )
    model.add(keras.layers.Dense(10, activation="softmax"))
    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice("learning_rate", [1.0, 1e-2, 1e-3, 1e-4])),
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=["accuracy"],
    )

    return model

In [ ]:
tuner = kt.Hyperband(
    mnist_model,
    objective='val_accuracy',
    max_epochs=10,
    directory='my_dir',
    project_name='minst_tuner_2'
)



stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20)

tuner.search_space_summary()

In [2]:
val_features, val_labels = pickle.load(open('val_data.pickle','rb'))
train_features,train_labels = pickle.load(open('train_data.pickle','rb'))
test_features, test_labels = pickle.load(open('test_data.pickle','rb'))
train_features = load_cifar.features_reshape(train_features)
val_features = load_cifar.features_reshape(val_features)
test_features = load_cifar.features_reshape(test_features)

In [ ]:
tuner.search(train_features, train_labels, epochs=10, validation_split=0.2, callbacks=[stop_early])

<h1>Training and testing</h1>
<h2>1.Print out validation accuracy after each training poch</h2>
<h2>2.Print out training time you spend on each epoch</h2>
<h2>3.Print out testing accuracy in the end</h2>